This notebook is dedicated to data preparation both for *model preparation* (see the XX notebook) and for *predictions* performed in my app.   
It outputs 5 differents things:

- one local dataset: local (saved in XXX)
- a hospital data set
- several datasets for models
- some dictionnaries used in my app

# 1. Data gathering

In [90]:
import pandas as pd
import os
import numpy as np

## 1.1 Data from the CMS website

### 1.1.1 The inpatient provider utilization and payment data

The main dataset: **"Medicare Provider Utilization and Payment Data: Inpatient"** for year 2017, which is the most recent year available:   
https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Medicare-Provider-Charge-Data/Inpatient

In [29]:
%%bash
wget https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Medicare-Provider-Charge-Data/Downloads/Inpatient_Data_2017_CSV.zip -nc -q 
unzip -u Inpatient_Data_2017_CSV.zip

Archive:  Inpatient_Data_2017_CSV.zip


In [ ]:
%%bash
git add MEDICARE_PROVIDER_CHARGE_INPATIENT_DRGALL_FY2017.CSV
git commit

In [30]:
%%bash
rm Inpatient_Data_2017_CSV.zip

### 1.1.2 The last *hospital compare* flat files package 


The hospital compare dataset (choice: most recent data)
https://data.medicare.gov/data/archives/hospital-compare

Pick up two important files:  
- the **general information file** (with hospital ownership, average ratings...) 
- the **Medicare Hospital Spending per Patient - Hospital.csv** file which gives an assessment of the spending per patient in a price-standardized and risk-adjusted way. https://www.medicare.gov/hospitalcompare/Data/Medicare-Spending-Beneficiary.html


In [36]:
%%bash
wget http://medicare.gov/download/HospitalCompare/2020/July/HOSArchive_Revised_Flatfiles_20200731.zip -nc -q 


In [37]:
if not os.path.exists('hospital_compare_data'):
    os.makedirs('hospital_compare_data')

In [38]:
%%bash
cp HOSArchive_Revised_Flatfiles_20200731.zip hospital_compare_data 
rm HOSArchive_Revised_Flatfiles_20200731.zip

In [39]:
%%bash
cd hospital_compare_data
unzip HOSArchive_Revised_Flatfiles_20200731.zip 
ls

Archive:  HOSArchive_Revised_Flatfiles_20200731.zip
  inflating: ASC_CCN_pr19q1_19q4.csv  
  inflating: ASC_Facility.csv        
  inflating: ASC_National.csv        
  inflating: ASC_NATIONAL_pr19q1_19q4.csv  
  inflating: ASC_State.csv           
  inflating: ASC_STATE_pr19q1_19q4.csv  
  inflating: CJR PY4 Quality Reporting_July 2020_Production File.csv  
  inflating: CMS_PSI_6_decimal_file.csv  
  inflating: Complications and Deaths - Hospital.csv  
  inflating: Complications and Deaths - National.csv  
  inflating: Complications and Deaths - State.csv  
  inflating: Footnote Crosswalk.csv  
  inflating: footnotes_deliver_19q1_19q4.csv  
  inflating: FY2018_Distribution_of_Net_Change_in_Base_Op_DRG_Payment_Amt_2019-11-22.csv  
  inflating: FY2018_Net_Change_in_Base_Op_DRG_Payment_Amt_2019-11-22.csv  
  inflating: FY2018_Percent_Change_in_Medicare_Payments_2019-11-22.csv  
  inflating: FY2018_Value_Based_Incentive_Payment_Amount_2019-11-22.csv  
  inflating: HCAHPS - Hospital.csv   

In [40]:
%%bash
cd hospital_compare_data
mv 'Hospital General Information.csv' Hospital_General_Information.csv
mv 'Medicare Hospital Spending per Patient - Hospital.csv' Medicare_Hospital_spending_per_patient_Hospital.csv

In [41]:
%%bash
cp hospital_compare_data/Hospital_General_Information.csv .
cp hospital_compare_data/Medicare_Hospital_spending_per_patient_Hospital.csv .

ls


Data wrangling.ipynb
Hospital_General_Information.csv
MEDICARE_PROVIDER_CHARGE_INPATIENT_DRGALL_FY2017.CSV
Medicare_Hospital_spending_per_patient_Hospital.csv
hospital_compare_data


In [42]:
%%bash
rm -r hospital_compare_data
ls

Data wrangling.ipynb
Hospital_General_Information.csv
MEDICARE_PROVIDER_CHARGE_INPATIENT_DRGALL_FY2017.CSV
Medicare_Hospital_spending_per_patient_Hospital.csv


In [43]:
%%bash
git add Hospital_General_Information.csv Medicare_Hospital_spending_per_patient_Hospital.csv
git commit

[master 7f34862] Add two additionnal files from CMS: Hospital_General_Information.csv and Medicare_Hospital_spending_per_patient_Hospital.csv
 2 files changed, 10034 insertions(+)
 create mode 100644 raw_inputs/Hospital_General_Information.csv
 create mode 100644 raw_inputs/Medicare_Hospital_spending_per_patient_Hospital.csv


## 1.2 "Local" Data

Download:

- a zipcode - HRR (hospital referral region) crosswalk table from the Dartmouth Atlas
https://atlasdata.dartmouth.edu/downloads/geography/ZipHsaHrr17.xls    
- a crosswalk between zipcodes and counties https://www.huduser.gov/portal/datasets/usps_crosswalk.html (done manually)  
- IRS revenue data at county level: https://www.irs.gov/pub/irs-soi/17incyallnoagi.csv
- census data: cc-est2017-alldata.csv from (

In [46]:
%%bash
wget https://atlasdata.dartmouth.edu/downloads/geography/ZipHsaHrr17.xls  -q -nc

In [44]:
#move from download
os.rename("/Users/camilledethe/downloads/ZIP_COUNTY_122017.xlsx", "ZIP_COUNTY_122017.xlsx")


In [47]:
%%bash
wget https://www.irs.gov/pub/irs-soi/17incyallnoagi.csv  -q -nc

In [49]:
os.rename("/Users/camilledethe/downloads/us-zip-code-latitude-and-longitude.csv", "us-zip-code-latitude-and-longitude.csv")

In [82]:
#move from download
os.rename("/Users/camilledethe/downloads/cc-est2017-alldata.csv", "cc-est2017-alldata.csv")


# 2. Data preparation


## 2.1 Preparation of the "local" dataset

### 2.1.1 Preparation of the basic structures: zipcode, longitude, latitude, county, HRR

In [53]:
zip_coun=pd.read_excel('ZIP_COUNTY_122017.xlsx',usecols=[0,1],skiprows=0,dtype={'zip':str,'county':str})

In [55]:
#Pb for a couple of zipcodes we have several counties, select just the first one
zip_coun=zip_coun.groupby('zip').county.agg(['first']).reset_index()

In [60]:
zip_coun.rename(columns={'first':'county'}, inplace=True)

In [63]:
long_lat=pd.read_csv('us-zip-code-latitude-and-longitude.csv',sep=';', dtype={'Zip':str}, 
                     usecols=['Zip','City','State','Latitude','Longitude'])

In [101]:
local_1=zip_coun.merge(long_lat, left_on='zip',right_on='Zip')

In [69]:
zip_hrr=pd.read_excel('ZipHsaHrr17.xls',dtype={'zipcode2017':str,'county':str})


In [102]:
local_2=zip_hrr.merge(local_1,left_on='zipcode2017',right_on='zip')

In [103]:
local_2=local_2[['zipcode2017','hrrnum', 'county','City','State','Latitude','Longitude']]

### 2.1.2 Adjunction of IRS data at county level

In [104]:
df2=pd.read_csv('17incyallnoagi.csv', usecols=['STATEFIPS','COUNTYFIPS','N00200','A00200','A00100','N1','N2'],dtype={'STATEFIPS':str,'COUNTYFIPS':str ,'CBSACODE':str})



In [105]:

df2['COUNTY_FULL_FIPS']=df2['STATEFIPS']+df2['COUNTYFIPS']

irs_2=df2.groupby('COUNTY_FULL_FIPS').sum().reset_index()

irs_2['average_AGI_c']=irs_2.A00100/irs_2.N1
irs_2['average_wage_c']=irs_2.A00200  /irs_2.N00200

irs_2=irs_2[['COUNTY_FULL_FIPS','average_AGI_c','average_wage_c']]

In [106]:
local_3=local_2.merge(irs_2, left_on='county',right_on='COUNTY_FULL_FIPS')

In [107]:
local_3.drop(columns=['COUNTY_FULL_FIPS'],inplace=True)

### 2.1.3 Adjunction of Census data at county level

In [108]:
dfc=pd.read_csv('cc-est2017-alldata.csv',encoding='latin-1', 
            usecols=['STATE','COUNTY','YEAR','AGEGRP','TOT_POP', 'BA_MALE', 'BA_FEMALE',  'H_MALE','H_FEMALE'],
            dtype= {'STATE':str,'COUNTY':str}   )


In [109]:
#Select only the lines for 2017
dfc=dfc[dfc['YEAR']==10]

In [110]:
#recreate the county
dfc['COUNTY_ID']=dfc['STATE']+dfc['COUNTY']


In [111]:
dfc['BA']=dfc['BA_MALE']+dfc['BA_FEMALE']
dfc['H']=dfc['H_MALE']+dfc['H_FEMALE']


In [112]:



dfc['over_65']=0
dfc.loc[dfc['AGEGRP']>13,'over_65']=dfc['TOT_POP']

# build the proportion of people above 65 ie agegrp 14,15,16,17,18
dfc=dfc.groupby('COUNTY_ID').agg(
                pop_tot=pd.NamedAgg(column='TOT_POP', aggfunc='max'),
                pop_ba=pd.NamedAgg(column='BA', aggfunc='max'),
                pop_h=pd.NamedAgg(column='H', aggfunc='max'),
                pop_over_65=pd.NamedAgg(column='over_65', aggfunc=np.sum ))
   

dfc=dfc.reset_index()

dfc['per_over_65']=dfc.pop_over_65/dfc.pop_tot
dfc['per_ba']=dfc.pop_ba/dfc.pop_tot
dfc['per_h']=dfc.pop_h/dfc.pop_tot

dfc=dfc[['COUNTY_ID','per_over_65','per_ba','per_h']]

In [114]:
local = local_3.merge(dfc,left_on='county',right_on='COUNTY_ID' )

## 2.2 Preparation of the hospital dataset

In [50]:
%%bash
ls

17incyallnoagi.csv
Data wrangling.ipynb
Hospital_General_Information.csv
MEDICARE_PROVIDER_CHARGE_INPATIENT_DRGALL_FY2017.CSV
Medicare_Hospital_spending_per_patient_Hospital.csv
ZIP_COUNTY_122017.xlsx
ZipHsaHrr17.xls
index.html?format=csv
us-zip-code-latitude-and-longitude.csv


In [92]:
local_3

,zipcode2017,hrrnum,county,City,State,Latitude,Longitude,average_AGI_c,average_wage_c
0,10001,303,36061,New York,NY,40.750742,-73.99653,226.146157,146.194817
1,10002,303,36061,New York,NY,40.717040,-73.98700,226.146157,146.194817
2,10003,303,36061,New York,NY,40.732509,-73.98935,226.146157,146.194817
3,10004,303,36061,New York,NJ,40.699226,-74.04118,226.146157,146.194817
4,10005,303,36061,New York,NY,40.706019,-74.00858,226.146157,146.194817
...,...,...,...,...,...,...,...,...,...
35994,99922,10,02198,Hydaburg,AK,55.209339,-132.82545,52.254701,44.950000
35995,99923,10,02198,Hyder,AK,55.941442,-130.05450,52.254701,44.950000
35996,99925,10,02198,Klawock,AK,55.555164,-133.07316,52.254701,44.950000
35997,99926,10,02198,Metlakatla,AK,55.123897,-131.56883,52.254701,44.950000


In [115]:
%%bash
git status

On branch master
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	../.DS_Store
	.DS_Store
	.ipynb_checkpoints/
	17incyallnoagi.csv
	Data wrangling.ipynb
	ZIP_COUNTY_122017.xlsx
	ZipHsaHrr17.xls
	cc-est2017-alldata.csv
	us-zip-code-latitude-and-longitude.csv

nothing added to commit but untracked files present (use "git add" to track)


In [116]:
%%bash
git ls-tree -r master --name-only



Hospital_General_Information.csv
MEDICARE_PROVIDER_CHARGE_INPATIENT_DRGALL_FY2017.CSV
Medicare_Hospital_spending_per_patient_Hospital.csv
